In [2]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import shutil
import random

from scipy import signal
from collections import OrderedDict

In [17]:
path = r'c:\Users\dhers\OneDrive - UBA\DLC\prueba_label_2\smooth_0\try_labelingDLC_Resnet50_STORM_2Oct14shuffle2_snapshot_900.h5'

df = pd.read_hdf(path)
scorer = df.columns.levels[0][0]
bodyparts = df.columns.levels[1].to_list()

df_raw = df.copy()
df

scorer    DLC_Resnet50_STORM_2Oct14shuffle2_snapshot_900              \
bodyparts                                          obj_1               
coords                                                 x           y   
0                                             217.828476  133.605682   
1                                             217.911606  133.641449   
2                                             217.993073  133.654526   
3                                             217.928711  133.677216   
4                                             217.948502  133.609283   
..                                                   ...         ...   
895                                           218.048630  133.233475   
896                                           218.171738  133.092514   
897                                           217.823441  133.319611   
898                                           217.325638  132.952866   
899                                           217.134918  132.707809   

scorer                                                               \
bodyparts                  obj_2                               nose   
coords    likelihood           x           y likelihood           x   
0           0.964640  484.968079  136.885910   0.962163  604.445862   
1           0.968036  485.034790  136.854279   0.959463  602.956421   
2           0.972242  485.000977  136.765106   0.956326  600.506836   
3           0.971661  485.052917  136.713028   0.956236  600.675171   
4           0.971781  485.055298  136.698639   0.958180  601.511963   
..               ...         ...         ...        ...         ...   
895         0.931858  484.273163  136.484955   0.950439  202.731323   
896         0.930851  484.306396  136.471954   0.950280  202.527344   
897         0.936866  484.330261  136.477768   0.952174  204.397400   
898         0.923255  484.314697  136.490570   0.951288  205.289734   
899         0.925533  484.308411  136.499084   0.951109  205.539597   

scorer                                        ...                         \
bodyparts                              L_ear  ...       body      tail_1   
coords              y likelihood           x  ... likelihood           x   
0           88.664230   0.945342  584.742432  ...   0.895273  596.825439   
1           76.554443   0.991210  585.924805  ...   0.888124  599.557739   
2           62.340645   0.947466  584.445740  ...   0.942344  595.908203   
3           51.426857   0.938307  585.179626  ...   0.912149  594.799255   
4           39.533306   0.919699  585.284851  ...   0.945490  593.590515   
..                ...        ...         ...  ...        ...         ...   
895        158.949585   0.881371  156.007156  ...   0.911939  100.209290   
896        158.496643   0.884174  155.111130  ...   0.926902  100.740730   
897        155.804565   0.884495  155.372452  ...   0.912233  100.803009   
898        152.466293   0.907526  156.921844  ...   0.871219  101.138321   
899        150.222687   0.926155  155.797394  ...   0.851610  101.821632   

scorer                                                               \
bodyparts                             tail_2                          
coords              y likelihood           x           y likelihood   
0          256.805359   0.933015  588.435303  317.164062   0.884303   
1          244.664093   0.941433  594.650330  305.439636   0.993319   
2          234.874115   0.928471  591.044800  301.362366   0.912296   
3          221.811295   0.946441  588.853210  285.576813   0.901532   
4          207.437317   0.886840  589.555420  271.838593   0.930745   
..                ...        ...         ...         ...        ...   
895        295.466797   0.921927  112.993492  359.467407   0.937825   
896        292.240967   0.968336  107.589767  357.366882   0.891820   
897        289.970245   0.946593  104.928032  354.048553   0.912491   
898        289.247345   0.938496  105.014244  353.381958   0.911225   
899     

In [18]:
# Try different filtering parameters
window, sigma, n_sigmas = 3, 1, 2
N = int(2 * n_sigmas * sigma + 1)

# Gaussian kernel
kernel = signal.windows.gaussian(N, sigma)
kernel = kernel / sum(kernel)

pad_width = (len(kernel) - 1) // 2

In [19]:
# Iterate over each body part
for point in bodyparts:

    mean = df[scorer, point, 'likelihood'].mean()
    std_dev = df[scorer, point, 'likelihood'].std()
    limit = max(0.1, mean - 0*std_dev)

    print(f'{point}\t{limit}')
    
    # Create mask where likelihood is below the threshold
    mask = df[scorer, point, 'likelihood'] < limit
    # Set 'x' and 'y' to NaN where likelihood is below the threshold
    df.loc[mask, (scorer, point, ['x', 'y'])] = np.nan

    # Iterate over x and y axes
    for axis in ['x', 'y']:
        # Access the specific axis column for this body part
        # column = df[scorer, point, axis]

        # Check if there are any non-NaN values left to interpolate
        if df[scorer, point, axis].notna().sum() > 1:
            # Interpolate using the pchip method, limit_area='inside' ensures interpolation only within valid data
            df[scorer, point, axis] = df[scorer, point, axis].interpolate(method='pchip', limit_area='inside')
            
            # Forward fill the remaining NaN values
            df[scorer, point, axis] = df[scorer, point, axis].ffill()
            
            # Apply median filter
            df[scorer, point, axis] = signal.medfilt(df[scorer, point, axis], kernel_size=window)
            
            # Pad the median filtered data to mitigate edge effects
            padded = np.pad(df[scorer, point, axis], pad_width, mode='edge')
            
            # Apply convolution
            smooth = signal.convolve(padded, kernel, mode='valid')
            
            # Trim the padded edges to restore original length
            df[scorer, point, axis] = smooth[:len(df[scorer, point, axis])]
        
            if 'obj' in point:
                df[scorer, point, axis] = df[scorer, point, axis].median()
            


df

L_ear	0.9416981339454651
R_ear	0.9281390309333801
body	0.9441000819206238
head	0.9104166030883789
neck	0.9260424971580505
nose	0.8370545506477356
obj_1	0.9489573836326599
obj_2	0.947649359703064
tail_1	0.9339337944984436
tail_2	0.9304155111312866
tail_3	0.9142758250236511


scorer    DLC_Resnet50_STORM_2Oct14shuffle2_snapshot_900              \
bodyparts                                          obj_1               
coords                                                 x           y   
0                                             217.656804  133.627798   
1                                             217.656804  133.627798   
2                                             217.656804  133.627798   
3                                             217.656804  133.627798   
4                                             217.656804  133.627798   
..                                                   ...         ...   
895                                           217.656804  133.627798   
896                                           217.656804  133.627798   
897                                           217.656804  133.627798   
898                                           217.656804  133.627798   
899                                           217.656804  133.627798   

scorer                                                               \
bodyparts                  obj_2                               nose   
coords    likelihood           x           y likelihood           x   
0           0.964640  484.346212  136.476921   0.962163  602.832119   
1           0.968036  484.346212  136.476921   0.959463  602.275034   
2           0.972242  484.346212  136.476921   0.956326  601.402153   
3           0.971661  484.346212  136.476921   0.956236  601.394562   
4           0.971781  484.346212  136.476921   0.958180  603.154011   
..               ...         ...         ...        ...         ...   
895         0.931858  484.346212  136.476921   0.950439  202.156744   
896         0.930851  484.346212  136.476921   0.950280  203.124320   
897         0.936866  484.346212  136.476921   0.952174  204.155176   
898         0.923255  484.346212  136.476921   0.951288  204.932420   
899         0.925533  484.346212  136.476921   0.951109  205.241112   

scorer                                        ...                         \
bodyparts                              L_ear  ...       body      tail_1   
coords              y likelihood           x  ... likelihood           x   
0           75.779952   0.945342  584.482849  ...   0.895273  178.897376   
1           71.714246   0.991210  584.520816  ...   0.888124  419.420598   
2           62.678261   0.947466  584.696703  ...   0.942344  564.274295   
3           51.408639   0.938307  585.002936  ...   0.912149  595.863435   
4           40.751275   0.919699  585.094083  ...   0.945490  595.943786   
..                ...        ...         ...  ...        ...         ...   
895        159.279724   0.881371  155.135081  ...   0.911939  100.829234   
896        157.742722   0.884174  155.355470  ...   0.926902  100.818177   
897        155.513983   0.884495  155.369354  ...   0.912233  100.899319   
898        152.939947   0.907526  155.372452  ...   0.871219  101.037465   
899        151.074728   0.926155  155.372452  ...   0.851610  101.120050   

scorer                                                               \
bodyparts                             tail_2                          
coords              y likelihood           x           y likelihood   
0           72.461309   0.933015  177.425153   90.563088   0.884303   
1          167.573083   0.941433  415.913203  209.935141   0.993319   
2          219.270917   0.928471  559.384131  276.306692   0.912296   
3          221.563584   0.946441  590.510537  282.491609   0.901532   
4          209.898549   0.886840  590.863002  272.421495   0.930745   
..                ...        ...         ...         ...        ...   
895        294.980240   0.921927  114.712117  360.209689   0.937825   
896        292.475877   0.968336  113.299142  359.595817   0.891820   
897        290.577331   0.946593  112.993492  359.467407   0.912491   
898        289.586997   0.938496  112.993492  359.467407   0.911225   
899     

In [20]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df[scorer]['nose']['x']))
fig.add_trace(go.Scatter(x=df_raw.index, y=df_raw[scorer]['nose']['x']))

In [21]:
# Save the updated DataFrame to an HDF5 file
out_path = r'c:\Users\dhers\OneDrive - UBA\DLC\prueba_label_2\smooth_0\try_labelingDLC_Resnet50_STORM_2Oct14shuffle2_snapshot_900.h5'
df.to_hdf(out_path, key='df', mode='w')